In [1]:
from pyspark import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql import SQLContext, Row  
from pyspark.sql import functions as F
sc =SparkContext()
spark = SparkSession(sc)
sqlContext=SQLContext(sc)

In [2]:
temps= sc.textFile("C:/Users/quartermaine/Documents/Big Data Analytics Labs/temperature-readings-tiny.csv")
parts_temp = temps.map(lambda l:l.split(";"))
tempReadings = parts_temp.map(lambda p: Row(station=p[0],  date=p[1], year=p[1].split("-")[0],month=p[1].split("-")[1], time=p[2],  Temp=float(p[3]), quality=p[4]))
#Inferring the schema and registering the DataFrame as atable
schemaTempReadings =  sqlContext.createDataFrame(tempReadings)
schemaTempReadings.registerTempTable("tempReadings")
schemaTempReadings.show()

+----+----------+-----+-------+-------+--------+----+
|Temp|      date|month|quality|station|    time|year|
+----+----------+-----+-------+-------+--------+----+
| 6.8|2013-11-01|   11|      G| 102170|06:00:00|2013|
| 3.8|2013-11-01|   11|      G| 102170|18:00:00|2013|
| 5.8|2013-11-02|   11|      G| 102170|06:00:00|2013|
|-1.1|2013-11-02|   11|      G| 102170|18:00:00|2013|
|-0.2|2013-11-03|   11|      G| 102170|06:00:00|2013|
| 5.6|2013-11-03|   11|      G| 102170|18:00:00|2013|
| 6.5|2013-11-04|   11|      G| 102170|06:00:00|2013|
| 5.1|2013-11-04|   11|      G| 102170|18:00:00|2013|
| 4.2|2013-11-05|   11|      G| 102170|06:00:00|2013|
| 3.2|2013-11-05|   11|      G| 102170|18:00:00|2013|
| 1.7|2013-11-06|   11|      G| 102170|06:00:00|2013|
| 0.9|2013-11-06|   11|      G| 102170|18:00:00|2013|
|-0.1|2013-11-07|   11|      G| 102170|06:00:00|2013|
| 0.1|2013-11-07|   11|      G| 102170|18:00:00|2013|
|-1.2|2013-11-08|   11|      G| 102170|06:00:00|2013|
| 5.3|2013-11-08|   11|     

In [7]:
precipitation = sc.textFile("C:/Users/quartermaine/Documents/Big Data Analytics Labs/precipitation-readings.csv")

parts_precip = precipitation.map(lambda a: a.split(';'))
precReadingsRow = parts_precip.map(lambda x: Row(station=x[0], date=x[1], year=int(x[1].split("-")[0]), month=int(x[1].split("-")[1]), time=x[2], precip=float(x[3]), quality=x[4]))
precReadingsRow = sqlContext.createDataFrame(precReadingsRow)
precReadingsRow.registerTempTable("precipReadings")
precReadingsRow.show()

+----------+-----+------+-------+-------+--------+----+
|      date|month|precip|quality|station|    time|year|
+----------+-----+------+-------+-------+--------+----+
|1995-08-01|    8|   0.0|      Y| 103100|00:00:00|1995|
|1995-08-01|    8|   0.0|      Y| 103100|01:00:00|1995|
|1995-08-01|    8|   0.0|      Y| 103100|02:00:00|1995|
|1995-08-01|    8|   0.0|      Y| 103100|03:00:00|1995|
|1995-08-01|    8|   0.0|      Y| 103100|04:00:00|1995|
|1995-08-01|    8|   0.0|      Y| 103100|05:00:00|1995|
|1995-08-01|    8|   0.0|      Y| 103100|06:00:00|1995|
|1995-08-01|    8|   0.0|      Y| 103100|07:00:00|1995|
|1995-08-01|    8|   0.0|      Y| 103100|08:00:00|1995|
|1995-08-01|    8|   0.0|      Y| 103100|09:00:00|1995|
|1995-08-01|    8|   0.0|      Y| 103100|10:00:00|1995|
|1995-08-01|    8|   0.0|      Y| 103100|11:00:00|1995|
|1995-08-01|    8|   0.0|      Y| 103100|12:00:00|1995|
|1995-08-01|    8|   0.0|      Y| 103100|13:00:00|1995|
|1995-08-01|    8|   0.0|      Y| 103100|14:00:0

In [23]:
sch=schemaTempReadings.groupby('station').agg(F.max('Temp').alias('maxTemp'))
sch1=sch.filter( (sch['maxTemp'] >25) & (sch['maxTemp'] <30) )

In [24]:
prec=precReadingsRow.groupby('station').agg(F.max('precip').alias('maxPrecip'))
prec1=prec.filter( (prec['maxPrecip'] >100) & (prec['maxPrecip'] <200) )

In [27]:
joinedSchema=sch1.join(prec1,'station')
joinedSchema.show()

+-------+-------+---------+
|station|maxTemp|maxPrecip|
+-------+-------+---------+
+-------+-------+---------+

